# HW1 (Q15-20)

### Q15
First, we use an artificial data set to study PLA. The data set is in https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_15_train.dat 

Each line of the data set contains one **(x<sub>n</sub>, *y*<sub>n</sub>)** with **x<sub>n</sub> ∈ R<sup>4</sup>**. The first 4 numbers of the line contains the components of *x*<sub>n</sub> orderly, the last number is *y*<sub>n</sub>.
Please initialize your algorithm with w = 0 and take sign(0)as -1. Please always remember to add x0 = 1 to each x.

What is the number of updates before the algorithm halts?

In [186]:
import random as rd
import numpy as np

In [505]:
#Load Data and Return Array 

def load_data(infile:'Data Path') -> array:
    X = []
    Y = []
    with open(infile) as f:
        for line in f:
            recs = line.split()
            x = [1] + [float(v) for v in recs[:-1]]
            X.append(tuple(x))
            Y.append(int(recs[-1]))
    return array(X), array(Y)

#Determine the sign 
def sign(x:int) -> int:
    if x <= 0:
        return -1
    else: 
        return 1


In [507]:
#Naive PLA
def naive_pla(X: array, Y: array, randomize = False, eta = 1) -> int:
    n = len(Y) #Number of Y
    d = len(X[0]) #Dimension of X
    W = zeros(d) #Intial W
    iteration = 0
    rand = range(n) #If randomize == False then go through the data by row
    
    if randomize: #If randomize == True then create a new order of row index
        idx = range(n)
        rand = rd.sample(idx, n)
    
    while True: 
        false_data = 0

        for i in rand:
             if Y[i] != sign(np.dot(W, X[i])): #If Y != sign(WX)
                W += eta*Y[i]*X[i]  #Update W
                iteration += 1 #Update iteration number
                false_data += 1
        if not false_data: #When there is no false data in this round of testing then break
            break
        
    return iteration
    

In [508]:
load_data('./hw1_15_train.dat.txt')
naive_pla(X,Y)  #Halt after 45 updates 

45

### Q16 
Implement a version of PLA by visiting examples in fixed, pre-determined random cycles throughout the algorithm. Run the algorithm on the data set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average number of updates before the algorithm halts?  

In [509]:
def naive_pla_randomize(X: array, Y: array, times: int, eta=1) -> float:
    iteration = 0
    for r in range(times): 
        rd.seed(r) #set a new seed
        iteration += naive_pla(X, Y, randomize = True, eta=eta)
        
    return iteration/times

In [510]:
naive_pla_randomize(X, Y, 2000) #39.624

39.624

### Q17
Implement a version of PLA by visiting examples in fixed, pre-determined random cycles throughout the algorithm, while changing the update rule to be **W<sub>t+1</sub> <- W<sub>t</sub> + η*y<sub>n(t)</sub>X<sub>n(t)</sub>** with **η** = 0.5. Note that your PLA in the previous Question corresponds to η=1. Please repeat your experiment for 2000 times, each with a different random seed. What is the average number of updates before the algorithm halts?

In [511]:
naive_pla_randomize(X, Y, 2000, eta = 0.5) #39.624 indiciating that eta does not change required iteration number

39.624

### Q18
Next, we play with the pocket algorithm. Modify your PLA in Question 16 to visit examples purely randomly, and then add the "pocket" steps to the algorithm. 

We will use https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_train.dat as the training data set **D**, and https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_test.dat as the test set for "verifying" the *g* returned by your algorithm (see lecture 4 about verifying). 

The sets are of the same format as the previous one. Run the pocket algorithm with a total of 50 updates on **D** , and verify the performance of **W<sub>pocket</sub>** using the test set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?

### 19 

Modify your algorithm in Question 18 to return **w** (the PLA vector after 50 updates) instead of **W<sub>pocket</sub>**  after 50 updates. Run the modified algorithm on **D**, and verify the performance using the test set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?


In [286]:
#Load Data
Xtrain, Ytrain = load_data('./hw1_18_train.dat.txt')
Xtest, Ytest = load_data('./hw1_18_test.dat.txt')

In [516]:
def test(X, Y, W):
    n = len(Y)
    ne = sum([1 for i in range(n) if sign(np.dot(W, X[i])) != Y[i]]) #number of errors
    errorRate = ne / float(n)
    return errorRate

def pocket_pla(X: array, Y: array, endpoint: int, randomize = False, eta = 1) -> array:
    n = len(Y) #Number of Y
    d = len(X[0]) #X dimension
    W = zeros(d) #Initial G
    Wp = zeros(d) #Wpocket 
    
    iteration = 0 
    rand = range(n) 
    
    if randomize:
        idx = range(n)
        rand = rd.sample(idx, n)
        
    error_g = test(X, Y, Wp) #Initial error generated by Wg*X
    
    while iteration != endpoint: #when iteration number has not reached set limitation 
        for i in rand:
            if Y[i] != sign(np.dot(W, X[i])):
                W  += eta*Y[i]*X[i]
                error = test(X, Y, W)
                if error < error_g: #if current error is smaller than the Wg's error 
                    error_g = error
                    Wp = W.copy() #Update Wg with W's weights (use W.copy() instead of W to avoid shared id confunsion)
                iteration += 1
            if iteration == endpoint:
                break
                
    return W, Wp


In [517]:
sumErrorW = 0
sumErrorWp = 0
k = 200
#Should be set to 2000 for the HW, but PLC is very slow. You can start with samller number before you jump to 2000
for r in range(k):
    rd.seed(r)
    W, Wp = pocket_pla(Xtrain, Ytrain, 50, randomize = True)
    sumErrorW += test(Xtest, Ytest, W)
    sumErrorWp += test(Xtest, Ytest, Wp)
    
meanErrorW = sumErrorW/k
meanErrorWp = sumErrorWp/k

0.36651000000000006 0.13259599999999996


In [522]:
#Q18
print('mean error of W pocket on the testing set is %f' %meanErrorWp) 
#Q19
print('mean error of W on the teseting set is %f' %meanErrorW) 

mean error of W pocket on the testing set is 0.129900
mean error of W on the teseting set is 0.366510


### Q20
Modify your algorithm in Question 18 to run for 100 updates instead of 50, and verify the performance of **W<sub>pocket</sub>** using the test set. Please repeat your experiment for 2000 times, each with a different random seed. What is the average error rate on the test set?

In [525]:
sumErrorWp100 = 0
k = 200
for r in range(k):
    rd.seed(r)
    W, Wg = pocket_pla(Xtrain, Ytrain, 100, randomize = True)
    sumErrorWp100 += test(Xtest, Ytest, Wp)
    
meanErrorWp100 = sumErrorWp100/k

In [526]:
#Q20
print('mean error of W on the teseting set with 100 updates is %f' %meanErrorWp100) 

mean error of W on the teseting set with 100 updates is 0.100000
